In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Loading Data

In [1]:

import json
import pandas as pd

In [2]:
#loading scores
with open(r'/content/drive/MyDrive/Level Data Datasets/2023 District 18 Scores.json') as scores_data:
    sc_json = json.load(scores_data)

    #assign dataframe
    ld_scores = pd.DataFrame(sc_json['SELECT s.id, s.districtId, sc.*\r\nFROM aim2.students s\r\nJOIN aim2.scores sc ON s.id = sc.studentId \r\nWHERE s.districtId = 18'])
    # we will be making many changes to ld_scores, let us save for later the original dataframe.
    # note: this cell takes a LONG time


In [3]:
    # we will be making many changes to ld_scores, let us save for later the original dataframe.
    # note: this cell takes a LONG time
    ld_scores_original = ld_scores.copy()

In [4]:
#loading benchmarks
with open(r'/content/drive/MyDrive/Level Data Datasets/benchmarks_202410011642.json') as benchmarks_data:
    benchmarks_json = json.load(benchmarks_data)
    ld_benchmarks = pd.DataFrame(benchmarks_json['benchmarks'])

In [5]:
#load schools
with open(r'/content/drive/MyDrive/Level Data Datasets/District 18 Anonymized Schools.json') as schools_data:
    schools_json = json.load(schools_data)
    #assign dataframe
    ld_schools = pd.DataFrame(schools_json['schools'])

In [6]:
#load vendor student usage
with open(r'/content/drive/MyDrive/Level Data Datasets/2023 District 18 Vendor Student Usage.json') as vendor_data:
    vendor_json = json.load(vendor_data)
    #assign dataframe
    ld_vendorUsage = pd.DataFrame(vendor_json['SELECT s.districtId, sc.studentId, sc.`year` , sc.vendorId, sc.active, sc.usageTypeId, sc.weeklyUsageMinutes \r\nFROM aim2.students s\r\nJOIN aim2.vendorStudentUsage sc ON s.id = sc.studentId \r\nWHERE s.districtId = 18 AND sc.`year` = 2023'])

In [7]:
#load student attributes
with open(r'/content/drive/MyDrive/Level Data Datasets/2023 District 18 Student Attributes.json') as student_attributes_data:
    student_attributes_json = json.load(student_attributes_data)
    ld_attributes = pd.DataFrame(student_attributes_json['SELECT s.districtId, sc.studentId, sc.`year` , sc.value \r\nFROM aim2.students s\r\nJOIN aim2.studentCustomAttributes sc ON s.id = sc.studentId \r\nWHERE s.districtId = 18'])

In [8]:
#load vendor usage types
with open(r'/content/drive/MyDrive/Level Data Datasets/District 18 Vendor Usage Types.json') as vendor_ref_data:
    vendor_ref_json = json.load(vendor_ref_data)
    ld_vendorTypes = pd.DataFrame(vendor_ref_json['vendorUsageTypes'])

# Vendor Inspection
Finding exactly what columns we need to include in our scores datatable

In [9]:
ld_vendorTypes['name'].unique()

array(['IXL Reading Non', 'IXL Reading Partial', 'IXL Reading User',
       'IXL Math Non User', 'IXL Math Partial ', 'IXL Math User',
       'MyOn R User', 'MyOn R Partial User', 'MyOn R Non User',
       'Freckle R User', 'Freckle R Partial User', 'Freckle R Non User',
       'Freckle M User', 'Freckle M Partial User', 'Freckle M Non User ',
       'Freckle Sci User', 'Freckle Sci Partial User',
       'Freckle Sci NonUser', 'Freckle SS User', 'Freckle SS PartialUser',
       'Freckle SS NonUser', 'IXL Science User', 'IXL Science Non User',
       'IXL Science Partial', 'IXL SS User', 'IXL SS Non',
       'IXL SS Partial', 'Freckle M Adaptive User',
       'Freckle M Adaptive Partial User', 'Freckle M Adaptive Non User',
       'Freckle M Target User', 'Freckle M Target Partial User',
       'Freckle M Target Non User', 'Beable ELA User',
       'Beable ELA Partial User', 'Beable Non User', 'AR User',
       'AR Partial User', 'AR Non User', 'Reflex M User',
       'Reflex M Partial 

In [10]:
#'Beable Non User' doesn't match the 'Beable ELA [usetype]' syntax, change to 'Beable ELA Non User'
ld_vendorTypes['name'] = ld_vendorTypes['name'].replace('Beable Non User', 'Beable ELA Non User')

In [11]:
ld_vendorTypes['usageTypeId'].value_counts()
#we need to cut down on this. Let us inspect to make certain that these repeat values are useless

,count
usageTypeId,
2,20
1,19
3,19
7,15
8,13
9,13
4,10
5,10
6,10


In [12]:
ld_vendorTypes[ld_vendorTypes['usageTypeId']==29]

,id,name,districtId,usageTypeId,roiBucketType,initiativeId,vendorId,utilBucketType,weeklyUsageMinutes,productId
158,5755,Freckle M Adaptive Partial User,18,29,loss,1014,137682,partialUser,0,0
164,5761,Freckle M Adaptive Partial User,18,29,loss,1014,137682,partialUser,0,0


In [13]:
def auditNameSymmetry(usageTypeId, df): #may expand this function to check any dataframe for symmetry in any column. for now, unneccessary
  df_subset = df[df['usageTypeId']==usageTypeId]['name']
  if (df_subset.value_counts().shape[0] != 1):
    return False
  return True

  #this function ensures that for a given dataframe and usagetypeid, the name is the same for all elements


In [14]:
for usageTypeId in (ld_vendorTypes['usageTypeId'].unique()):
  print(auditNameSymmetry(usageTypeId, ld_vendorTypes)) #great, we know that extra values are redundant. By inspection, they only differ by initiativeId

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


Alright, let us now trim the fat on the vendorTypes. Let us make a new dataframe.

In [15]:
ld_trimmedVendorTypes = ld_vendorTypes[['usageTypeId', 'name','utilBucketType','id']]
ld_trimmedVendorTypes.head()

,usageTypeId,name,utilBucketType,id
0,1,IXL Reading Non,nonUser,4
1,2,IXL Reading Partial,partialUser,5
2,3,IXL Reading User,fullUser,6
3,4,IXL Math Non User,nonUser,7
4,5,IXL Math Partial,partialUser,8


In [16]:
ld_trimmedVendorTypes = ld_trimmedVendorTypes.drop_duplicates(subset=['usageTypeId'], keep='first')
ld_trimmedVendorTypes = ld_trimmedVendorTypes.reset_index(drop=True)
ld_trimmedVendorTypes['usageTypeId'].value_counts()

,count
usageTypeId,
1,1
23,1
27,1
26,1
28,1
29,1
30,1
31,1
32,1


Great. Now we have to find out all the columns we need to add. Really, we need to find all the product names. Let Gemini do this, and verify results.

In [17]:
# prompt: from ld_trimmedVendorTypes name column, find the name of the associated product by removing the key words 'Partial', 'Non User', 'NonUser', or 'User'

import re

def extract_product_name(name):
  """
  Extracts the product name from a vendor usage type name by removing
  keywords like 'Partial', 'Non User', 'NonUser', or 'User'.
  """
  name = re.sub(r'\b(Partial|Non User|NonUser|User|Non|PartialUser)\b', '', name)
  name = name.strip()
  return name

# Apply the function to the 'name' column and create a new column 'product_name'
ld_trimmedVendorTypes['product_name'] = ld_trimmedVendorTypes['name'].apply(extract_product_name)

# Print the updated DataFrame to verify
print(ld_trimmedVendorTypes[['name', 'product_name']])


                               name        product_name
0                   IXL Reading Non         IXL Reading
1               IXL Reading Partial         IXL Reading
2                  IXL Reading User         IXL Reading
3                 IXL Math Non User            IXL Math
4                 IXL Math Partial             IXL Math
5                     IXL Math User            IXL Math
6                       MyOn R User              MyOn R
7               MyOn R Partial User              MyOn R
8                   MyOn R Non User              MyOn R
9                    Freckle R User           Freckle R
10           Freckle R Partial User           Freckle R
11               Freckle R Non User           Freckle R
12                   Freckle M User           Freckle M
13           Freckle M Partial User           Freckle M
14              Freckle M Non User            Freckle M
15                 Freckle Sci User         Freckle Sci
16         Freckle Sci Partial User         Frec

This looks great. Now, let's make a column to associate usage type with a number 0-2. Let Gemini do this too.

In [18]:
ld_trimmedVendorTypes

,usageTypeId,name,utilBucketType,id,product_name
0,1,IXL Reading Non,nonUser,4,IXL Reading
1,2,IXL Reading Partial,partialUser,5,IXL Reading
2,3,IXL Reading User,fullUser,6,IXL Reading
3,4,IXL Math Non User,nonUser,7,IXL Math
4,5,IXL Math Partial,partialUser,8,IXL Math
5,6,IXL Math User,fullUser,9,IXL Math
6,7,MyOn R User,fullUser,853,MyOn R
7,8,MyOn R Partial User,partialUser,854,MyOn R
8,9,MyOn R Non User,nonUser,855,MyOn R
9,10,Freckle R User,fullUser,1130,Freckle R


In [19]:
# prompt: Make a new column for ld_trimmedVendorTypes that associates utilBucketType with an integer. 0 will be associated with 'nonUser', 1 will be associated with 'partialUser', 2 will be associated with 'fullUser', and -1 will be associated with any other utilBucketType value.

def map_util_bucket_type(util_bucket_type):
  """Maps utilBucketType to an integer."""
  if util_bucket_type == 'nonUser':
    return 0
  elif util_bucket_type == 'partialUser':
    return 1
  elif util_bucket_type == 'fullUser':
    return 2
  else:
    return -1

# Apply the function to create the new column
ld_trimmedVendorTypes['productUse_int'] = ld_trimmedVendorTypes['utilBucketType'].apply(map_util_bucket_type)

# Print the updated DataFrame to verify
#print(ld_trimmedVendorTypes[['utilBucketType', 'productUse_int']])
# Looks good, let's not clutter

In [20]:
ld_trimmedVendorTypes

,usageTypeId,name,utilBucketType,id,product_name,productUse_int
0,1,IXL Reading Non,nonUser,4,IXL Reading,0
1,2,IXL Reading Partial,partialUser,5,IXL Reading,1
2,3,IXL Reading User,fullUser,6,IXL Reading,2
3,4,IXL Math Non User,nonUser,7,IXL Math,0
4,5,IXL Math Partial,partialUser,8,IXL Math,1
5,6,IXL Math User,fullUser,9,IXL Math,2
6,7,MyOn R User,fullUser,853,MyOn R,2
7,8,MyOn R Partial User,partialUser,854,MyOn R,1
8,9,MyOn R Non User,nonUser,855,MyOn R,0
9,10,Freckle R User,fullUser,1130,Freckle R,2


In [21]:
venProductList = ld_trimmedVendorTypes['product_name'].unique()
venProductList
# venProductList = ['IXL Reading', 'IXL Math', 'MyOn R', 'Freckle R', 'Freckle M', 'Freckle Sci', 'Freckle SS', 'IXL Science', 'IXL SS', 'Freckle M Adaptive', 'Freckle M Target', 'Beable ELA', 'AR User', 'Reflex M', 'Frax M']

array(['IXL Reading', 'IXL Math', 'MyOn R', 'Freckle R', 'Freckle M',
       'Freckle Sci', 'Freckle SS', 'IXL Science', 'IXL SS',
       'Freckle M Adaptive', 'Freckle M Target', 'Beable ELA', 'AR',
       'Reflex M', 'Frax M'], dtype=object)

We want to add a number of columns to our ld_scores dataframe. First, we want to add and populate all the product use columns. Second, we want to one hot encode the attributes that we have data for. The first one is harder and more important, let's start there.

In [22]:
ld_vendorUsage.head()

,districtId,studentId,year,vendorId,active,usageTypeId,weeklyUsageMinutes
0,18,1480117,2023,140472,0,39,4
1,18,1480117,2023,11333,0,6,0
2,18,1480117,2023,72609,0,3,0
3,18,1480118,2023,140472,0,39,4
4,18,1480118,2023,11333,0,6,0


We need to associate each item here with the scores data set by studentID. Then, we need to light up the rows product with its productUse_int from trimmed vendor types. First, let us trim vendorUsage for this purpose, and append the necessary columns to the scores dataset.

TODO for later, should we start caring about weekly usage minutes?

In [23]:
ld_trimmedVendorUsage = ld_vendorUsage[['studentId', 'usageTypeId']]
ld_trimmedVendorUsage.head()

,studentId,usageTypeId
0,1480117,39
1,1480117,6
2,1480117,3
3,1480118,39
4,1480118,6


In [24]:
# prompt: For every item in venProductList, add a column to ld_scores of the name of the product+" user", and populate that column with only 0s.

for product_name in venProductList:
  ld_scores["user_" + product_name] = 0

Before we make a bunch of changes to ld_scores, why don't we take out all the columns we don't need? We will also sort by studentId so any search algorithms we do can use binary search

In [25]:
# TODO, take out test scores that are useless
ld_scores_original['studentId'].value_counts()

,count
studentId,
1483818,19
1802905,19
1490116,19
1487272,19
1803268,19
...,...
1492164,1
1492170,1
1804160,1


In [26]:
hold_on=ld_vendorUsage['studentId'].value_counts()
hold_on[hold_on==4]

,count
studentId,
1488821,4
1488823,4
1802641,4
1802640,4
1485705,4
...,...
1484853,4
1484779,4
1484780,4


In [43]:
len(ld_vendorUsage['studentId'].unique())

11654

In [27]:
ld_vendorUsage[ld_vendorUsage['studentId']==1488823]

,districtId,studentId,year,vendorId,active,usageTypeId,weeklyUsageMinutes
42138,18,1488823,2023,11333,0,6,0
42139,18,1488823,2023,72609,0,2,0
42140,18,1488823,2023,140472,0,37,120
42141,18,1488823,2023,140574,0,40,0


In [54]:
ld_scores[ld_scores['studentId']==1488823]

,id,districtId,studentId,studentLevel,year,measurementTypeId,subgroup_specialEd,subgroup_lunchStatus,subgroup_gender,subgroup_ell,...,user_Freckle Sci,user_Freckle SS,user_IXL Science,user_IXL SS,user_Freckle M Adaptive,user_Freckle M Target,user_Beable ELA,user_AR,user_Reflex M,user_Frax M
88847,7692595,18,1488823,3,2023,18,3,0,2,2,...,0,0,0,0,0,0,0,0,0,0
88839,7692587,18,1488823,3,2023,10,3,0,2,2,...,0,0,0,0,0,0,0,0,0,0
88845,7692593,18,1488823,3,2023,16,3,0,2,2,...,0,0,0,0,0,0,0,0,0,0
88844,7692592,18,1488823,3,2023,15,3,0,2,2,...,0,0,0,0,0,0,0,0,0,0
88843,7692591,18,1488823,3,2023,14,3,0,2,2,...,0,0,0,0,0,0,0,0,0,0
88842,7692590,18,1488823,3,2023,13,3,0,2,2,...,0,0,0,0,0,0,0,0,0,0
88841,7692589,18,1488823,3,2023,12,3,0,2,2,...,0,0,0,0,0,0,0,0,0,0
88840,7692588,18,1488823,3,2023,11,3,0,2,2,...,0,0,0,0,0,0,0,0,0,0
88846,7692594,18,1488823,3,2023,17,3,0,2,2,...,0,0,0,0,0,0,0,0,0,0
88838,7692586,18,1488823,3,2023,8,3,0,2,2,...,0,0,0,0,0,0,0,0,0,0


In [29]:
len(ld_scores['studentId'].unique())

11633

Ok, so we have a discrepancy. We have lists of measurementTypeId data from ld_scores, but also from ld_vendor_use. Let us for now assume that the data from both these tables are true for now. However, we will check to see if they conflict for any product. We will construct an ld_scores prime dataframe to hold each unique studentID, which we will then populate with our '- user' values

In [30]:
to_drop = ['ticket', 'subgroup_TCAPELALevel', 'scoreDate', 'subgroup_ethnicity']
ld_scores = ld_scores.drop(columns=to_drop)
#ld_scores.head()

In [31]:
ld_scores = ld_scores.sort_values('studentId')


In [32]:
ld_scores.head()

,id,districtId,studentId,studentLevel,year,measurementTypeId,subgroup_specialEd,subgroup_lunchStatus,subgroup_gender,subgroup_ell,...,user_Freckle Sci,user_Freckle SS,user_IXL Science,user_IXL SS,user_Freckle M Adaptive,user_Freckle M Target,user_Beable ELA,user_AR,user_Reflex M,user_Frax M
0,3268686,18,1480117,11,2023,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4933778,18,1480117,11,2023,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5441518,18,1480117,11,2023,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,7263146,18,1480117,11,2023,6,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,7510656,18,1480117,11,2023,7,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
ld_scoresPrime = ld_scores.drop_duplicates(subset=['studentId'], keep='first')
ld_scoresPrime = ld_scoresPrime.reset_index(drop=True)
ld_scoresPrime = ld_scoresPrime.drop(columns=['measurementTypeId'], axis=1)

In [61]:
ld_scoresPrime[ld_scoresPrime['studentId']==1488823]['user_IXL Reading']

,user_IXL Reading
7238,0


In [55]:
ld_vendorUsage[ld_vendorUsage['studentId']==1488823]

,districtId,studentId,year,vendorId,active,usageTypeId,weeklyUsageMinutes
42138,18,1488823,2023,11333,0,6,0
42139,18,1488823,2023,72609,0,2,0
42140,18,1488823,2023,140472,0,37,120
42141,18,1488823,2023,140574,0,40,0


In [56]:
ld_trimmedVendorTypes

,usageTypeId,name,utilBucketType,id,product_name,productUse_int
0,1,IXL Reading Non,nonUser,4,IXL Reading,0
1,2,IXL Reading Partial,partialUser,5,IXL Reading,1
2,3,IXL Reading User,fullUser,6,IXL Reading,2
3,4,IXL Math Non User,nonUser,7,IXL Math,0
4,5,IXL Math Partial,partialUser,8,IXL Math,1
5,6,IXL Math User,fullUser,9,IXL Math,2
6,7,MyOn R User,fullUser,853,MyOn R,2
7,8,MyOn R Partial User,partialUser,854,MyOn R,1
8,9,MyOn R Non User,nonUser,855,MyOn R,0
9,10,Freckle R User,fullUser,1130,Freckle R,2


In [78]:
def translateMeasureIdToTarget(row, reference, target):
  #print(row)
  studentId = row['studentId']
  usageTypeId = row['usageTypeId']
  productItem = ld_trimmedVendorTypes[ld_trimmedVendorTypes['usageTypeId'] == usageTypeId]
  productName = productItem['product_name'].iloc[0]
  productUse_int = productItem['productUse_int'].iloc[0]
  rowToPopulate = -1
  try:
    columnToPopulate = "user_"+productName
    rowToPopulate = target[target['studentId'] == studentId].index[0]
  except IndexError:
    print(f"Student ID {studentId} not found in ld_scoresPrime.")

  if rowToPopulate == -1:
    return
  target.loc[rowToPopulate, columnToPopulate] = productUse_int
  #print('Student ID: ' + str(studentId) + ' Product: ' + productName + ' ProductUse_int: ' + str(productUse_int))



In [80]:
ld_scoresPrime[ld_scoresPrime['studentId']==1483668]

,id,districtId,studentId,studentLevel,year,subgroup_specialEd,subgroup_lunchStatus,subgroup_gender,subgroup_ell,subGroup_bottom25,...,user_Freckle Sci,user_Freckle SS,user_IXL Science,user_IXL SS,user_Freckle M Adaptive,user_Freckle M Target,user_Beable ELA,user_AR,user_Reflex M,user_Frax M


In [86]:
for currRow in range(0, ld_trimmedVendorUsage.shape[0]):
  translateMeasureIdToTarget(row=ld_trimmedVendorUsage.loc[currRow], reference=ld_trimmedVendorTypes, target=ld_scoresPrime)


Student ID 1480143 not found in ld_scoresPrime.
Student ID 1480143 not found in ld_scoresPrime.
Student ID 1480143 not found in ld_scoresPrime.
Student ID 1480148 not found in ld_scoresPrime.
Student ID 1480148 not found in ld_scoresPrime.
Student ID 1480148 not found in ld_scoresPrime.
Student ID 1480157 not found in ld_scoresPrime.
Student ID 1480157 not found in ld_scoresPrime.
Student ID 1480157 not found in ld_scoresPrime.
Student ID 1480185 not found in ld_scoresPrime.
Student ID 1480185 not found in ld_scoresPrime.
Student ID 1480185 not found in ld_scoresPrime.
Student ID 1480185 not found in ld_scoresPrime.
Student ID 1480185 not found in ld_scoresPrime.
Student ID 1480196 not found in ld_scoresPrime.
Student ID 1480196 not found in ld_scoresPrime.
Student ID 1480196 not found in ld_scoresPrime.
Student ID 1480220 not found in ld_scoresPrime.
Student ID 1480220 not found in ld_scoresPrime.
Student ID 1480220 not found in ld_scoresPrime.
Student ID 1480225 not found in ld_score

In [88]:
#so how many students did we not cover?
primeVendorUsage = ld_trimmedVendorUsage.drop_duplicates(subset=['studentId'], keep='first')
primeVendorUsage = primeVendorUsage.reset_index(drop=True)
scores_missing = 0
for currRow in range(0, primeVendorUsage.shape[0]):
  if ld_scoresPrime[ld_scoresPrime['studentId']==(primeVendorUsage.loc[currRow]['studentId'])].empty:
    scores_missing += 1
print("Students missing in scores data (ideally is 0): " + str(scores_missing))

products_missing = 0

for currRow in range(0, ld_scoresPrime.shape[0]):
  if primeVendorUsage[primeVendorUsage['studentId']==(ld_scoresPrime.loc[currRow]['studentId'])].empty:
    products_missing += 1
print("Students without product data (not unexpected to be nonzero): " + str(products_missing))

Students missing in scores data (ideally is 0): 689
Students without product data (not unexpected to be nonzero): 668


In [94]:
user_cols = ld_scoresPrime.filter(regex='^user_').columns
for column in user_cols:
  print(ld_scoresPrime[column].value_counts())


user_IXL Reading
2    5518
0    3800
1    2315
Name: count, dtype: int64
user_IXL Math
2    5420
0    3894
1    2319
Name: count, dtype: int64
user_MyOn R
0    11633
Name: count, dtype: int64
user_Freckle R
0    11633
Name: count, dtype: int64
user_Freckle M
0    11633
Name: count, dtype: int64
user_Freckle Sci
0    11633
Name: count, dtype: int64
user_Freckle SS
0    11633
Name: count, dtype: int64
user_IXL Science
0    10831
1      602
2      200
Name: count, dtype: int64
user_IXL SS
0    11261
1      259
2      113
Name: count, dtype: int64
user_Freckle M Adaptive
0    11633
Name: count, dtype: int64
user_Freckle M Target
0    11633
Name: count, dtype: int64
user_Beable ELA
0    11633
Name: count, dtype: int64
user_AR
0    9005
2    1683
1     945
Name: count, dtype: int64
user_Reflex M
0    10468
1     1004
2      161
Name: count, dtype: int64
user_Frax M
0    11633
Name: count, dtype: int64


It looks like we can eliminate the following products, since no students in the districts use them:
 - Frax M
 - Freckle M
 - Freckle M Target
 - Freckle M Adaptive
 - Beable ELA
 - Freckle R
 - Freckle Sci
 - Freckle SS
 - MyOn R

 So, the vendor products we **can** start to look at follow:
 - Reflex M
 - AR
 - IXL SS
 - IXL Science
 - IXL Math
 - IXL Reading